In [1]:
# %%capture
# !pip install gdown --force-reinstall
# !gdown --id 1TEOjzSBN6UZc1WQwb_huEhdFl1XyZel4
# !unzip KDEF_CROPPED_ALIGNED.zip

In [2]:
# %%capture
# !gdown --id 1Bd87admxOZvbIOAyTkGEntsEz3fyMt7H

In [3]:
# %%capture
# !pip install mlxtend
# !pip install dlib
# !pip install scikit-image

In [4]:
import torch
torch.multiprocessing.set_start_method('spawn')

In [5]:
import imageio
import matplotlib.pyplot as plt
from mlxtend.image import extract_face_landmarks
import cv2

# MagFace

In [6]:
import torch
from torch import nn
# from torchvision.models.utils import load_state_dict_from_url

__all__ = ['iresnet18', 'iresnet34', 'iresnet50', 'iresnet100']


def conv3x3(in_planes, out_planes, stride=1, groups=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, groups=groups, bias=False, dilation=dilation)


def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)


class IBasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=1,
                 base_width=64, dilation=1):
        super(IBasicBlock, self).__init__()
        if groups != 1 or base_width != 64:
            raise ValueError(
                'BasicBlock only supports groups=1 and base_width=64')
        if dilation > 1:
            raise NotImplementedError(
                "Dilation > 1 not supported in BasicBlock")
        # Both self.conv1 and self.downsample layers downsample the input when stride != 1
        self.bn1 = nn.BatchNorm2d(inplanes, eps=2e-05, momentum=0.9)
        self.conv1 = conv3x3(inplanes, planes)
        self.bn2 = nn.BatchNorm2d(planes, eps=2e-05, momentum=0.9)
        self.prelu = nn.PReLU(planes)
        self.conv2 = conv3x3(planes, planes, stride)
        self.bn3 = nn.BatchNorm2d(planes, eps=2e-05, momentum=0.9)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x
        out = self.bn1(x)
        out = self.conv1(out)
        out = self.bn2(out)
        out = self.prelu(out)
        out = self.conv2(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity

        return out


class IResNet(nn.Module):
    fc_scale = 7 * 7

    def __init__(self, block, layers, num_classes=512, zero_init_residual=False,
                 groups=1, width_per_group=64, replace_stride_with_dilation=None):
        super(IResNet, self).__init__()

        self.inplanes = 64
        self.dilation = 1
        if replace_stride_with_dilation is None:
            replace_stride_with_dilation = [False, False, False]
        if len(replace_stride_with_dilation) != 3:
            raise ValueError("replace_stride_with_dilation should be None "
                             "or a 3-element tuple, got {}".format(replace_stride_with_dilation))
        self.groups = groups
        self.base_width = width_per_group
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=3, stride=1, padding=1,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(self.inplanes, eps=2e-05, momentum=0.9)
        self.prelu = nn.PReLU(self.inplanes)
        self.layer1 = self._make_layer(block, 64, layers[0], stride=2)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2,
                                       dilate=replace_stride_with_dilation[0])
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2,
                                       dilate=replace_stride_with_dilation[1])
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2,
                                       dilate=replace_stride_with_dilation[2])
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.bn2 = nn.BatchNorm2d(
            512 * block.expansion, eps=2e-05, momentum=0.9)
        self.dropout = nn.Dropout2d(p=0.4, inplace=True)
        self.fc = nn.Linear(512 * block.expansion * self.fc_scale, num_classes)
        self.features = nn.BatchNorm1d(num_classes, eps=2e-05, momentum=0.9)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(
                    m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, (nn.BatchNorm2d, nn.GroupNorm)):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, IBasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False):
        downsample = None
        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion,
                               eps=2e-05, momentum=0.9),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, self.groups,
                            self.base_width, previous_dilation))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=self.groups,
                                base_width=self.base_width, dilation=self.dilation))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.prelu(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.bn2(x)
        x = self.dropout(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        x = self.features(x)

        return x


def _iresnet(arch, block, layers, pretrained, progress, **kwargs):
    model = IResNet(block, layers, **kwargs)
    # if pretrained:
    # state_dict = load_state_dict_from_url(model_urls[arch],
    #                                        progress=progress)
    # model.load_state_dict(state_dict)
    return model


# def iresnet18(pretrained=False, progress=True, **kwargs):
#     return _iresnet('iresnet18', IBasicBlock, [2, 2, 2, 2], pretrained, progress,
#                     **kwargs)


# def iresnet34(pretrained=False, progress=True, **kwargs):
#     return _iresnet('iresnet34', IBasicBlock, [3, 4, 6, 3], pretrained, progress,
#                     **kwargs)


# def iresnet50(pretrained=False, progress=True, **kwargs):
#     return _iresnet('iresnet50', IBasicBlock, [3, 4, 14, 3], pretrained, progress,
#                     **kwargs)


def iresnet100(pretrained=False, progress=True, **kwargs):
    return _iresnet('iresnet100', IBasicBlock, [3, 13, 30, 3], pretrained, progress, **kwargs)

In [7]:
from collections import OrderedDict
import sys

def clean_dict_inf(model, state_dict):
    _state_dict = OrderedDict()
    for k, v in state_dict.items():
        # # assert k[0:1] == 'features.module.'
        # new_k = 'features.'+'.'.join(k.split('.')[2:])
        new_k = '.'.join(k.split('.')[2:])
        if new_k in model.state_dict().keys() and v.size() == model.state_dict()[new_k].size():
            _state_dict[new_k] = v
        
        new_kk = '.'.join(k.split('.')[1:])
        if new_kk in model.state_dict().keys() and v.size() == model.state_dict()[new_kk].size():
            _state_dict[new_kk] = v
            
    num_model = len(model.state_dict().keys())
    num_ckpt = len(_state_dict.keys())
    if num_model != num_ckpt:
        sys.exit("=> Not all weights loaded, model params: {}, loaded params: {}".format(
            num_model, num_ckpt))
    return _state_dict
    
    
def load_dict_inf(model, resume, cpu_mode=False):
    if os.path.isfile(resume):
        print(f'=> loading pth from {resume} ...')
        if cpu_mode:
            checkpoint = torch.load(resume, map_location=torch.device("cpu"))
        else:
            checkpoint = torch.load(resume)
        _state_dict = clean_dict_inf(model, checkpoint['state_dict'])
        model_dict = model.state_dict()
        model_dict.update(_state_dict)
        model.load_state_dict(model_dict)
        # delete to release more space
        del checkpoint
        del _state_dict
    else:
        sys.exit("=> No checkpoint found at '{}'".format(resume))
    return model


# Dataset Preparation

In [8]:
# !rm -rf /media/soroushh/Storage2/matrices
# !mkdir -p /media/soroushh/Storage2/matrices/training
# !mkdir -p /media/soroushh/Storage2/matrices/evaluation

# magface = iresnet100(pretrained=False, num_classes=512)
# magface = load_dict_inf(magface, "./magface_epoch_00025.pth")
# magface = magface.to("cuda")
# magface.eval()

# def get_landmarks(image_paths, image_size):
#     database = {}
#     for image_path in tqdm.tqdm(image_paths):
#         filename = image_path.split("/")[-1].split(".")[0]
#         if not bool(re.match(r"\w{2}\d{2}\w{2}\w+", filename)):
#             continue

#         name = re.findall(r"(\w{2}\d{2})\w{2}\w+", filename)[0]
#         pose_label = re.findall(r"\w{2}\d{2}\w{2}(\w+)", filename)[0]
#         emotion_label = re.findall(r"\w{2}\d{2}(\w{2})\w+", filename)[0]

#         # img = cv2.imread(image_path)
#         img = cv2.imread(image_path.replace("CROPPED_ALIGNED", "KDEF")) # get real images from KDEF for landmarks
#         img = cv2.resize(img, (image_size, image_size))
#         img = cv2.copyMakeBorder(img, 10, 10, 10, 10, cv2.BORDER_CONSTANT)
#         pose_img = get_pose_image(img, image_size)

#         if pose_img is None:
#             continue

#         person_db = database.get(name, [])
#         person_db.append((emotion_label, pose_label, pose_img, image_path))
#         database[name] = person_db
        
#     return database

# def get_pose_image(img, image_size):
#     landmarks = extract_face_landmarks(img)

#     if np.all(landmarks == 0) or landmarks is None:
#         return None

#     landmarks[:, 0][landmarks[:, 0] >= image_size] = image_size - 1
#     landmarks[:, 0][landmarks[:, 0] < 0] = 0
#     landmarks[:, 1][landmarks[:, 1] >= image_size] = image_size - 1
#     landmarks[:, 1][landmarks[:, 1] < 0] = 0

#     # landmarks = landmarks[[30, 40, 46, 48, 56]]
#     # print(landmarks.shape)

#     pose_img = np.zeros((image_size, image_size))
#     pose_img[landmarks[:, 1], landmarks[:, 0]] = 1
#     pose_img = pose_img[:, :, np.newaxis]
#     pose_img = pose_img[landmarks[:, 1].min():landmarks[:, 1].max(), landmarks[:, 0].min():landmarks[:, 0].max()]
#     pose_img = cv2.resize(pose_img, (image_size, image_size))
#     pose_img = np.where(pose_img > 0.5, 1, 0)

#     # plt.imshow(pose_img)
#     # plt.show()

#     pose_img = pose_img[:, :, np.newaxis]

#     return pose_img

# def get_magface_embedding(image_path):
#     img = cv2.imread(image_path)

#     if img.shape[:2] != [112, 112]:
#         img = cv2.resize(img, (112, 112))

#     img = img[np.newaxis] / 255.
#     img = torch.Tensor(img).to("cuda").to(torch.float32)
#     img = img.permute(0, 3, 1, 2)
#     embedding = magface(img)
#     embedding = embedding.detach().cpu().numpy()
#     embedding = np.squeeze(embedding)

#     return embedding

# def prepare_final_database(database):
#     db_keys = list(database.keys())

#     final_database = []
#     for person1_name in db_keys:
#         person1 = database[person1_name]
#         for triple_to_reconstruct_index, triple_to_reconstruct in enumerate(person1):
#             pose_img_to_reconstruct = triple_to_reconstruct[2]

#             available_poses_input = []
#             available_poses_output = []
#             for person2_name in db_keys:
#                 person2 = database[person2_name]
#                 available_poses_input = list(filter(lambda index: person2[index][0] == triple_to_reconstruct[0], range(len(person2))))
#                 available_poses_output = list(filter(lambda index: person2[index][0] == triple_to_reconstruct[0] and person2[index][1] == triple_to_reconstruct[1], range(len(person2))))

#                 for triple_to_change_input_index in available_poses_input:
#                     for triple_to_change_output_index in available_poses_output:
#                         final_database.append((person1_name, triple_to_reconstruct_index, person2_name, triple_to_change_input_index, triple_to_change_output_index))

#                 if len(final_database) >= 30000:
#                     break

#             if len(final_database) >= 30000:
#                 break

#         if len(final_database) >= 30000:
#             break
            
#     return final_database

# def save_files(database, final_database, type_):
#     for idx in tqdm.tqdm(range(len(final_database))):
#         tup = final_database[idx]

#         if os.path.isfile(f'/media/soroushh/Storage2/matrices/{type_}/{tup[0]}_{tup[1]}_{tup[2]}_{tup[3]}_{tup[4]}_pose_img_to_reconstruct.npy') and os.path.isfile(f'/media/soroushh/Storage2/matrices/{type_}/{tup[0]}_{tup[1]}_{tup[2]}_{tup[3]}_{tup[4]}_embedding_input.npy') and os.path.isfile(f'/media/soroushh/Storage2/matrices/{type_}/{tup[0]}_{tup[1]}_{tup[2]}_{tup[3]}_{tup[4]}_embedding_output.npy'):
#             continue

#         pose_img_to_reconstruct = database[tup[0]][tup[1]][2]
#         image_path = database[tup[2]][tup[3]][3]
#         embedding_input = get_magface_embedding(image_path)
#         image_path = database[tup[2]][tup[4]][3]
#         embedding_output = get_magface_embedding(image_path)

#         with open(f'/media/soroushh/Storage2/matrices/{type_}/{tup[0]}_{tup[1]}_{tup[2]}_{tup[3]}_{tup[4]}_pose_img_to_reconstruct.npy', 'wb') as f:
#             np.save(f, pose_img_to_reconstruct)

#         with open(f'/media/soroushh/Storage2/matrices/{type_}/{tup[0]}_{tup[1]}_{tup[2]}_{tup[3]}_{tup[4]}_embedding_input.npy', 'wb') as f:
#             np.save(f, embedding_input)

#         with open(f'/media/soroushh/Storage2/matrices/{type_}/{tup[0]}_{tup[1]}_{tup[2]}_{tup[3]}_{tup[4]}_embedding_output.npy', 'wb') as f:
#             np.save(f, embedding_output)

#     with open(f'/media/soroushh/Storage2/database_{type_}.pickle', 'wb') as handle:
#         pickle.dump(database, handle, protocol=pickle.HIGHEST_PROTOCOL)


# people_paths = glob.glob(os.path.join("./CROPPED_ALIGNED/*"))
# people_paths = np.array(people_paths)
# np.random.shuffle(people_paths)
# people_paths = people_paths.tolist()

# index = int(0.8 * len(people_paths))
# trainset = people_paths[:index]
# evalset = people_paths[index:]

# image_paths = [img_path for person_path in trainset for img_path in glob.glob(person_path + "/*.JPG")]
# database = get_landmarks(image_paths, 112)
# final_database = prepare_final_database(database)
# save_files(database, final_database, "training")

# image_paths = [img_path for person_path in evalset for img_path in glob.glob(person_path + "/*.JPG")]
# database = get_landmarks(image_paths, 112)
# final_database = prepare_final_database(database)
# save_files(database, final_database, "evaluation")

In [9]:
import glob
import numpy as np
import torch
import os
import tqdm
import re
import pickle
import random


class CustomDataSet(torch.utils.data.Dataset):

    def __init__(self, type_="train"):
        self.type_ = type_
        
        all_files = list(set([re.findall(r"(.+)_(\d+)_(.+)_(\d+)_(\d+)_.+", filename)[0] for filename in os.listdir(f"/media/soroushh/Storage2/matrices/training")]))
        if self.type_ == "train":
            all_files = all_files[:int((2/3) * len(all_files))]
        else:
            all_files = all_files[int((2/3) * len(all_files)):]
        
        self.final_database = []
        for person1_name, triple_to_reconstruct_index, person2_name, triple_to_change_input_index, triple_to_change_output_index in all_files:
            triple_to_reconstruct_index = int(triple_to_reconstruct_index)
            triple_to_change_input_index = int(triple_to_change_input_index)
            triple_to_change_output_index = int(triple_to_change_output_index)
            self.final_database.append((person1_name, triple_to_reconstruct_index, person2_name, triple_to_change_input_index, triple_to_change_output_index))
            
        with open(f'/media/soroushh/Storage2/database_training.pickle', 'rb') as handle:
            self.database = pickle.load(handle)
        
        ids = list(self.database.keys())
        self.ids_dict = {id:idx for idx, id in enumerate(list(self.database.keys()))}
        self.emotions_dict = {emo:idx for idx, emo in enumerate(np.unique([triplet[0] for id in ids for triplet in self.database[id]]).tolist())}
        self.poses_dict = {pose:idx for idx, pose in enumerate(np.unique([triplet[1] for id in ids for triplet in self.database[id]]).tolist())}

    def __len__(self):
        return len(self.final_database)
        
    def __getitem__(self, idx):
        tup = self.final_database[idx]
        
        with open(f'/media/soroushh/Storage2/matrices/training/{tup[0]}_{tup[1]}_{tup[2]}_{tup[3]}_{tup[4]}_pose_img_to_reconstruct.npy', 'rb') as f:
            pose_img_to_reconstruct = np.load(f)
            
        with open(f'/media/soroushh/Storage2/matrices/training/{tup[0]}_{tup[1]}_{tup[2]}_{tup[3]}_{tup[4]}_embedding_input.npy', 'rb') as f:
            embedding_input = np.load(f)
            
        with open(f'/media/soroushh/Storage2/matrices/training/{tup[0]}_{tup[1]}_{tup[2]}_{tup[3]}_{tup[4]}_embedding_output.npy', 'rb') as f:
            embedding_output = np.load(f)
            
        expected_pose_label = self.database[tup[2]][tup[4]][1]
        input_id = tup[2]
        input_emo = self.database[tup[2]][tup[3]][0]
        
        while True:
            available_negative_choices = list(filter(lambda item: item[2] != input_id, self.final_database))
            if len(available_negative_choices) == 0:
                continue
                
            neg_tup = random.choice(available_negative_choices)
            break
            
        with open(f'/media/soroushh/Storage2/matrices/training/{neg_tup[0]}_{neg_tup[1]}_{neg_tup[2]}_{neg_tup[3]}_{neg_tup[4]}_embedding_input.npy', 'rb') as f:
            negative_embedding_input = np.load(f)
            
        return pose_img_to_reconstruct, embedding_input, embedding_output, negative_embedding_input

In [10]:
train_dataset = CustomDataSet(type_="train")
val_dataset = CustomDataSet(type_="val")

In [11]:
len(train_dataset), len(val_dataset)

(20001, 10001)

In [12]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=256, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=256, shuffle=True)

# Network Arch.

In [13]:
def img_to_patch(x, patch_size, flatten_channels=True):
    """
    Inputs:
        x - torch.Tensor representing the image of shape [B, C, H, W]
        patch_size - Number of pixels per dimension of the patches (integer)
        flatten_channels - If True, the patches will be returned in a flattened format
                           as a feature vector instead of a image grid.
    """
    B, C, H, W = x.shape
    x = x.reshape(B, C, H//patch_size, patch_size, W//patch_size, patch_size)
    x = x.permute(0, 2, 4, 1, 3, 5) # [B, H', W', C, p_H, p_W]
    x = x.flatten(1,2)              # [B, H'*W', C, p_H, p_W]
    if flatten_channels:
        x = x.flatten(2,4)          # [B, H'*W', C*p_H*p_W]
    return x


class AttentionBlock(nn.Module):

    def __init__(self, embed_dim, hidden_dim, num_heads, dropout=0.0):
        """
        Inputs:
            embed_dim - Dimensionality of input and attention feature vectors
            hidden_dim - Dimensionality of hidden layer in feed-forward network
                         (usually 2-4x larger than embed_dim)
            num_heads - Number of heads to use in the Multi-Head Attention block
            dropout - Amount of dropout to apply in the feed-forward network
        """
        super().__init__()

        self.layer_norm_1 = nn.LayerNorm(embed_dim)
        self.attn = nn.MultiheadAttention(embed_dim, num_heads)
        self.layer_norm_2 = nn.LayerNorm(embed_dim)
        self.linear = nn.Sequential(
            nn.Linear(embed_dim, hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, embed_dim),
            nn.Dropout(dropout)
        )


    def forward(self, x):
        inp_x = self.layer_norm_1(x)
        x = x + self.attn(inp_x, inp_x, inp_x)[0]
        x = x + self.linear(self.layer_norm_2(x))
        return x


class VisionTransformer(nn.Module):

    def __init__(self, embed_dim, hidden_dim, num_channels, num_heads, num_layers, num_classes, patch_size, num_patches, dropout=0.0):
        """
        Inputs:
            embed_dim - Dimensionality of the input feature vectors to the Transformer
            hidden_dim - Dimensionality of the hidden layer in the feed-forward networks
                         within the Transformer
            num_channels - Number of channels of the input (3 for RGB)
            num_heads - Number of heads to use in the Multi-Head Attention block
            num_layers - Number of layers to use in the Transformer
            num_classes - Number of classes to predict
            patch_size - Number of pixels that the patches have per dimension
            num_patches - Maximum number of patches an image can have
            dropout - Amount of dropout to apply in the feed-forward network and
                      on the input encoding
        """
        super().__init__()

        self.patch_size = patch_size

        # Layers/Networks
        self.input_layer = nn.Linear(num_channels*(patch_size**2), embed_dim)
        self.transformer = nn.Sequential(*[AttentionBlock(embed_dim, hidden_dim, num_heads, dropout=dropout) for _ in range(num_layers)])
        self.mlp_head = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, num_classes)
        )
        self.dropout = nn.Dropout(dropout)

        # Parameters/Embeddings
        self.cls_token = nn.Parameter(torch.randn(1,1,embed_dim))
        self.pos_embedding = nn.Parameter(torch.randn(1,1+num_patches,embed_dim))


    def forward(self, x):
        # Preprocess input
        x = img_to_patch(x, self.patch_size)
        B, T, _ = x.shape
        x = self.input_layer(x)

        # Add CLS token and positional encoding
        cls_token = self.cls_token.repeat(B, 1, 1)
        x = torch.cat([cls_token, x], dim=1)
        x = x + self.pos_embedding[:,:T+1]

        # Apply Transforrmer
        x = self.dropout(x)
        x = x.transpose(0, 1)
        x = self.transformer(x)

        # Perform classification prediction
        cls = x[0]
        out = self.mlp_head(cls)
        # out = cls
        
        return out

In [14]:
import math
import torch.nn.functional as F


# class Reshape(torch.nn.Module):
    
#     def __init__(self, *args):
#         super(Reshape, self).__init__()
#         self.shape = args

#     def forward(self, x):
#         return x.view(self.shape)

    
# def scaled_dot_product(q, k, v, mask=None):
#     d_k = q.size()[-1]
#     attn_logits = torch.matmul(q, k.transpose(-2, -1))
#     attn_logits = attn_logits / math.sqrt(d_k)
#     if mask is not None:
#         attn_logits = attn_logits.masked_fill(mask == 0, -9e15)
        
#     attention = F.softmax(attn_logits, dim=-1)
#     values = torch.matmul(attention, v)
    
#     return values, attention


# class MultiheadAttention(torch.nn.Module):

#     def __init__(self, input_dim, embed_dim, num_heads):
#         super().__init__()
#         assert embed_dim % num_heads == 0, "Embedding dimension must be 0 modulo number of heads."

#         self.embed_dim = embed_dim
#         self.num_heads = num_heads
#         self.head_dim = embed_dim // num_heads

#         # Stack all weight matrices 1...h together for efficiency
#         # Note that in many implementations you see "bias=False" which is optional
#         self.qkv_proj = torch.nn.Linear(input_dim, 3*embed_dim)
#         self.o_proj = torch.nn.Linear(embed_dim, embed_dim)

#         self._reset_parameters()

#     def _reset_parameters(self):
#         # Original Transformer initialization, see PyTorch documentation
#         torch.nn.init.xavier_uniform_(self.qkv_proj.weight)
#         self.qkv_proj.bias.data.fill_(0)
#         torch.nn.init.xavier_uniform_(self.o_proj.weight)
#         self.o_proj.bias.data.fill_(0)

#     def forward(self, x, mask=None, return_attention=False):
#         batch_size, seq_length, embed_dim = x.size()
#         qkv = self.qkv_proj(x)

#         # Separate Q, K, V from linear output
#         qkv = qkv.reshape(batch_size, seq_length, self.num_heads, 3*self.head_dim)
#         qkv = qkv.permute(0, 2, 1, 3) # [Batch, Head, SeqLen, Dims]
#         q, k, v = qkv.chunk(3, dim=-1)

#         # Determine value outputs
#         values, attention = scaled_dot_product(q, k, v, mask=mask)
#         values = values.permute(0, 2, 1, 3) # [Batch, SeqLen, Head, Dims]
#         values = values.reshape(batch_size, seq_length, embed_dim)
#         o = self.o_proj(values)

#         if return_attention:
#             return o, attention
#         else:
#             return o


class EmbeddingGeneratorDecoder(torch.nn.Module):
    
    def __init__(self):
        super(EmbeddingGeneratorDecoder, self).__init__()

        # self.downsample1 = torch.nn.Conv1d(2, 8, 3, stride=1, padding=1)
        # self.downsample2 = torch.nn.Conv1d(8, 64, 3, stride=1, padding=1)
        
        # self.multihead_attention = MultiheadAttention(input_dim=128, embed_dim=128, num_heads=4)
        
        self.vit = VisionTransformer(**{
                                        'embed_dim': 128,
                                        'hidden_dim': 256,
                                        'num_heads': 4,
                                        'num_layers': 4,
                                        'patch_size': 8,
                                        'num_channels': 1,
                                        'num_patches': 32,
                                        'num_classes': 512,
                                        'dropout': 0.4
                                    })
        
        self.normalizer = nn.LayerNorm(1024)
        
#         self.upsample1 = torch.nn.Conv1d(64, 8, 3, stride=1, padding=1)
#         self.upsample2 = torch.nn.Conv1d(8, 1, 3, stride=1, padding=1)
        
#         self.upsample_block = torch.nn.Upsample(scale_factor=2)
        
        # self.batchnorm1 = torch.nn.BatchNorm1d(1)
        # self.batchnorm8 = torch.nn.BatchNorm1d(8)
        # self.batchnorm64 = torch.nn.BatchNorm1d(64)
        
        # self.features = torch.nn.Sequential(
        #     torch.nn.Linear(512, 512), 
        #     torch.nn.BatchNorm1d(512),
        # )
        
#     def feature_extraction_downsample(self, x): # n, 2, 512
#         skip_connections = []
        
#         x = self.downsample1(x) # n, 8, 512
#         # x = self.batchnorm8(x)
#         x = F.dropout(x, p=0.4)
#         x = F.relu(x)
#         skip_connections.append(x)
#         x = F.max_pool1d(x, kernel_size=2) # n, 8, 256 
#         # x = F.avg_pool1d(x, kernel_size=2) # n, 8, 256 
        
#         x = self.downsample2(x) # n, 64, 256
#         # x = self.batchnorm64(x)
#         x = F.dropout(x, p=0.4)
#         x = F.relu(x)
#         skip_connections.append(x)
#         x = F.max_pool1d(x, kernel_size=2) # n, 64, 128
#         # x = F.avg_pool1d(x, kernel_size=2) # n, 8, 256
        
#         return x, skip_connections
    
#     def feature_extraction_upsample(self, x, skip_connections): # n, 64, 128
#         skip_connections = list(reversed(skip_connections))
        
#         x = self.upsample_block(x) # n, 64, 256
#         x = x + skip_connections[0]
#         x = self.upsample1(x) # n, 8, 256
#         # x = self.batchnorm8(x)
#         x = F.dropout(x, p=0.4)
#         x = F.relu(x)
        
#         x = self.upsample_block(x) # n, 8, 512
#         x = x + skip_connections[1]
#         x = self.upsample2(x) # n, 1, 512
#         # x = self.batchnorm1(x)
#         x = F.dropout(x, p=0.4)
#         x = F.relu(x)
        
#         return x
    
    def forward(self, emb1, emb2):
#         emb1 = emb1.unsqueeze(1)
#         emb2 = emb2.unsqueeze(1)
        
#         comb = torch.cat([emb1, emb2], 1)
#         # comb = comb.unsqueeze(1)
#         comb, skip_connections = self.feature_extraction_downsample(comb)
#         # att_comb = self.multihead_attention(comb)
#         att_comb = comb
#         att_comb = self.feature_extraction_upsample(att_comb, skip_connections)
#         att_comb = att_comb.squeeze(1)
#         embedding = self.features(att_comb)
        
        comb = torch.cat([emb1, emb2], 1)
        comb = self.normalizer(comb)
        comb = torch.reshape(comb, (-1, 32, 32))
        comb = comb.unsqueeze(1)
        embedding = self.vit(comb)

        # comb = comb.unsqueeze(1)
        # comb, skip_connections = self.feature_extraction_downsample(comb)
        # # att_comb = self.multihead_attention(comb)
        # att_comb = comb
        # att_comb = self.feature_extraction_upsample(att_comb, skip_connections)
        # att_comb = att_comb.squeeze(1)
        
        # embedding = self.features(att_comb)
        
        return embedding


class ConvAutoencoder(torch.nn.Module):
    
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
        
        self.decoder_emb_generator = EmbeddingGeneratorDecoder()
        
        self.encoder_layer1 = torch.nn.Conv2d(1, 4, 3, stride=1, padding=1)
        self.encoder_layer2 = torch.nn.Conv2d(4, 16, 3, stride=1, padding=1)
        self.encoder_layer3 = torch.nn.Conv2d(16, 64, 3, stride=1, padding=1)
        self.encoder_layer4 = torch.nn.Conv2d(64, 128, 3, stride=1, padding=1)
        self.encoder_layer5 = torch.nn.Conv2d(128, 256, 3, stride=1, padding=1)
        self.encoder_layer6 = torch.nn.Conv2d(256, 512, 3, stride=1, padding=1)
        
        self.upsample2 = torch.nn.Upsample(scale_factor=2, mode='nearest')
        self.upsample3 = torch.nn.Upsample(scale_factor=3, mode='nearest')
        self.upsample4 = torch.nn.Upsample(scale_factor=4, mode='nearest')
        
        self.decoder_layer1 = torch.nn.Conv2d(512, 256, 2, stride=3, padding=2)
        self.decoder_layer2 = torch.nn.Conv2d(256, 128, 3, stride=1, padding=0)
        self.decoder_layer3 = torch.nn.Conv2d(128, 64, 3, stride=1, padding=1)
        self.decoder_layer4 = torch.nn.Conv2d(64, 16, 3, stride=1, padding=1)
        self.decoder_layer5 = torch.nn.Conv2d(16, 4, 3, stride=1, padding=1)
        self.decoder_layer6 = torch.nn.Conv2d(4, 1, 3, stride=1, padding=1)
        
        # self.bn8 = torch.nn.BatchNorm2d(8)
        # self.bn64 = torch.nn.BatchNorm2d(64)
        # self.bn256 = torch.nn.BatchNorm2d(256)
        # self.bn512 = torch.nn.BatchNorm2d(512)
        
    def encoder(self, x): # 1, 112, 112
        x = self.encoder_layer1(x) # 4, 112, 112
        x = F.dropout(x, p=0.4)
        # x = self.bn8(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2) # 4, 56, 56
        
        x = self.encoder_layer2(x) # 16, 56, 56
        x = F.dropout(x, p=0.4)
        # x = self.bn64(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2) # 16, 28, 28

        x = self.encoder_layer3(x) # 64, 28, 28
        x = F.dropout(x, p=0.4)
        # x = self.bn256(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2) # 64, 14, 14
        
        x = self.encoder_layer4(x) #128, 14, 14
        x = F.dropout(x, p=0.4)
        # x = self.bn512(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2) # 128, 7, 7
        
        x = self.encoder_layer5(x) #256, 7, 7
        x = F.dropout(x, p=0.4)
        # x = self.bn512(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2) # 256, 3, 3
        
        x = self.encoder_layer6(x) #512, 3, 3
        x = F.dropout(x, p=0.4)
        # x = self.bn512(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2) # 512, 1, 1
        
        x = torch.reshape(x, (-1, 512))
        
        return x
        
    def decoder_pose_reconstructor(self, x):
        x = torch.reshape(x, (-1, 512, 1, 1))
        
        x = self.upsample4(x) # 512, 4, 4
        x = self.decoder_layer1(x) # 256, 3, 3
        x = F.dropout(x, p=0.4)
        # x = self.bn256(x)
        x = F.relu(x)
        
        x = self.upsample3(x) # 256, 9, 9
        x = self.decoder_layer2(x) # 128, 7, 7
        x = F.dropout(x, p=0.4)
        # x = self.bn256(x)
        x = F.relu(x)
        
        x = self.upsample2(x) # 128, 14, 14
        x = self.decoder_layer3(x) # 64, 14, 14
        x = F.dropout(x, p=0.4)
        # x = self.bn256(x)
        x = F.relu(x)
        
        x = self.upsample2(x) # 64, 28, 28
        x = self.decoder_layer4(x) # 16, 28, 28
        x = F.dropout(x, p=0.4)
        # x = self.bn64(x)
        x = F.relu(x)
        
        x = self.upsample2(x) # 16, 56, 56
        x = self.decoder_layer5(x) # 4, 56, 56
        x = F.dropout(x, p=0.4)
        # x = self.bn8(x)
        x = F.relu(x)
        
        x = self.upsample2(x) # 4, 112, 112
        x = self.decoder_layer6(x) # 1, 112, 112
        x = torch.sigmoid(x)
        
        return x
        
    def forward(self, pose_img, magface_embedding):
        coded = self.encoder(pose_img)
        reconstructed_pose = self.decoder_pose_reconstructor(coded)
        
        # Feature fusion
        generated_embedding = self.decoder_emb_generator(coded, magface_embedding)
        
        # fc_out = self.fc_head(generated_embedding)
        
#         id_predictions = self.id_head(fc_out)
#         id_predictions = F.softmax(id_predictions, dim=1)
        id_predictions = None
        
#         emo_predictions = self.emotion_head(fc_out)
#         emo_predictions = F.softmax(emo_predictions, dim=1)
        emo_predictions = None
        
        return reconstructed_pose, generated_embedding, (id_predictions, emo_predictions)

# Train phase

In [15]:
net = ConvAutoencoder()
net = net.to("cuda")

state_dict = torch.load("./augmentor.pt")
net.load_state_dict(state_dict)

criterion_bce = torch.nn.BCELoss(reduction='mean')
criterion_mse = torch.nn.MSELoss()
# criterion_huber = torch.nn.HuberLoss()
# criterion_mae = torch.nn.L1Loss()
# criterion_cross = torch.nn.CrossEntropyLoss()
criterion_triplet_loss = nn.TripletMarginLoss(margin=1.0, p=2)

optimizer = torch.optim.Adam(net.parameters(), lr=1e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=5, min_lr=1e-6)

In [ ]:
import datetime


NUM_EPOCHS = 300

historical_loss = []
historical_val_loss = []

best_loss = np.inf
for epoch in range(NUM_EPOCHS):
    # training
    losses1 = []
    losses2 = []
    # losses3 = []
    # losses4 = []
    losses = []
    for data in tqdm.tqdm(train_dataloader):
        pose_img = data[0]
        pose_img = pose_img.permute(0, 3, 1, 2)
        pose_img = pose_img.to("cuda").to(torch.float32)
        
        input_emb = data[1]
        input_emb = input_emb.to("cuda").to(torch.float32)
        
        output_emb = data[2]
        output_emb = output_emb.to("cuda").to(torch.float32)
        
        negative_embedding = data[3]
        negative_embedding = negative_embedding.to("cuda").to(torch.float32)
        
        # expected_pose_label, input_id, input_emo = data[3:]
        # expected_pose_label = expected_pose_label.to("cuda")
        # input_id = input_id.to("cuda")
        # input_emo = input_emo.to("cuda")
        
        optimizer.zero_grad()
        
        reconstructed_pose, generated_emb, (id_predictions, emo_predictions) = net(pose_img, input_emb)
        
        loss1 = criterion_bce(reconstructed_pose, pose_img)
        losses1.append(loss1.item())
        
        # loss2 = criterion_mse(generated_emb, output_emb)
        loss2 = criterion_triplet_loss(generated_emb, output_emb, negative_embedding)
        losses2.append(loss2.item())
        
#         loss3 = criterion_cross(id_predictions, input_id)
#         losses3.append(loss3.item())
        
#         loss4 = criterion_cross(emo_predictions, input_emo)
#         losses4.append(loss4.item())
        
        loss = loss1 + loss2 #+ loss3 + loss4
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        
    # validation
    with torch.no_grad():
        val_losses1 = []
        val_losses2 = []
        # val_losses3 = []
        # val_losses4 = []
        val_losses = []
        for val_data in tqdm.tqdm(val_dataloader):
            pose_img = val_data[0]
            pose_img = pose_img.permute(0, 3, 1, 2)
            pose_img = pose_img.to("cuda").to(torch.float32)
            
            input_emb = val_data[1]
            input_emb = input_emb.to("cuda").to(torch.float32)
            
            output_emb = val_data[2]
            output_emb = output_emb.to("cuda").to(torch.float32)
            
            negative_embedding = val_data[3]
            negative_embedding = negative_embedding.to("cuda").to(torch.float32)
            
            # expected_pose_label, input_id, input_emo = val_data[3:]
            # expected_pose_label = expected_pose_label.to("cuda")
            # input_id = input_id.to("cuda")
            # input_emo = input_emo.to("cuda")
            
            reconstructed_pose, generated_emb, (id_predictions, emo_predictions) = net(pose_img, input_emb)
            
            val_loss1 = criterion_bce(reconstructed_pose, pose_img)
            val_losses1.append(val_loss1.item())
            
            # val_loss2 = criterion_mse(generated_emb, output_emb)
            val_loss2 = criterion_triplet_loss(generated_emb, output_emb, negative_embedding)
            val_losses2.append(val_loss2.item())
            
#             val_loss3 = criterion_cross(id_predictions, input_id)
#             val_losses3.append(val_loss3.item())
            
#             val_loss4 = criterion_cross(emo_predictions, input_emo)
#             val_losses4.append(val_loss4.item())
            
            val_loss = val_loss1 + val_loss2 #+ val_loss3 + val_loss4
            val_losses.append(val_loss.item())
            
    scheduler.step(val_loss)
    
    current_lr = optimizer.param_groups[0]['lr']
    
    print("Epoch", epoch+1, "/", NUM_EPOCHS, 
          "Loss", round(np.mean(losses), 4), f"({round(np.mean(losses1), 4)}, {round(np.mean(losses2), 4)})", 
          "Val_Loss", round(np.mean(val_losses), 4), f"({round(np.mean(val_losses1), 4)}, {round(np.mean(val_losses2), 4)})", 
          "LR", current_lr)
    
    # print("Loss", np.mean(losses), f"({np.mean(losses1)}, {np.mean(losses2)}, {np.mean(losses3)}, {np.mean(losses4)})")
    # print("Val_Loss", np.mean(val_losses), f"({np.mean(val_losses1)}, {np.mean(val_losses2)}, {np.mean(val_losses3)}, {np.mean(val_losses4)})")
    # historical_loss.append((np.mean(losses), np.mean(losses1), np.mean(losses2), np.mean(losses3), np.mean(losses4)))
    # historical_val_loss.append((np.mean(val_losses), np.mean(val_losses1), np.mean(val_losses2), np.mean(val_losses3), np.mean(val_losses4)))
    historical_loss.append((np.mean(losses), np.mean(losses1), np.mean(losses2)))
    historical_val_loss.append((np.mean(val_losses), np.mean(val_losses1), np.mean(val_losses2)))
    
    if historical_val_loss[-1][0] < best_loss:
        best_loss = historical_val_loss[-1][0]
        torch.save(net.state_dict(), "./augmentor.pt")
        print(f"Model saved @ {datetime.datetime.now()}, Loss = {round(historical_loss[-1][0], 4)}, Val_Loss = {round(historical_val_loss[-1][0], 4)}")

100%|██████████| 40/40 [13:34<00:00, 20.37s/it]


Epoch 1 / 300 Loss 0.7688 (0.2335, 0.5353) Val_Loss 0.1472 (0.1208, 0.0264) LR 0.001
Model saved @ 2022-02-03 12:19:26.902091, Loss = 0.7688, Val_Loss = 0.1472


100%|██████████| 40/40 [12:46<00:00, 19.15s/it]


Epoch 2 / 300 Loss 0.1261 (0.1065, 0.0196) Val_Loss 0.1122 (0.0979, 0.0143) LR 0.001
Model saved @ 2022-02-03 12:57:59.835312, Loss = 0.1261, Val_Loss = 0.1122


100%|██████████| 40/40 [11:45<00:00, 17.64s/it]


...

100%|██████████| 40/40 [10:15<00:00, 15.38s/it]


Epoch 40 / 300 Loss 0.0677 (0.0622, 0.0054) Val_Loss 0.0658 (0.0623, 0.0036) LR 0.00025
Model saved @ 2022-02-04 10:00:40.411237, Loss = 0.0677, Val_Loss = 0.0658


100%|██████████| 40/40 [10:29<00:00, 15.74s/it]


Epoch 41 / 300 Loss 0.0662 (0.0621, 0.004) Val_Loss 0.0676 (0.0622, 0.0054) LR 0.000125


 18%|█▊        | 14/79 [04:20<19:09, 17.69s/it]

In [ ]:
print("Loss", round(loss.item(), 4), "Val_Loss", round(val_loss.item(), 4), "LR", current_lr)

In [ ]:
# torch.save(net.state_dict(), "./augmentor.pt")

# Evaluation phase

In [ ]:
net = ConvAutoencoder()
net = net.to("cuda")
state_dict = torch.load("./augmentor.pt")
net.load_state_dict(state_dict)
net.eval()

In [ ]:
with torch.no_grad():
    for data in train_dataloader:
        pose_img = data[0]
        pose_img = pose_img.permute(0, 3, 1, 2)
        pose_img = pose_img.to("cuda").to(torch.float32)
        input_emb = data[1]
        input_emb = input_emb.to("cuda").to(torch.float32)
        output_emb = data[2]
        output_emb = output_emb.to("cuda").to(torch.float32)
        
        features = net.encoder(pose_img)
        
        reconstructed_data = net.decoder_pose_reconstructor(features)
        
        reconstructed_data = reconstructed_data.permute(0, 2, 3, 1).cpu().numpy()
        orig_data = pose_img.permute(0, 2, 3, 1).cpu().numpy()
        
        for i in range(reconstructed_data.shape[0]):
            plt.imshow(orig_data[i, :, :, 0])
            plt.show()
            plt.imshow(reconstructed_data[i, :, :, 0])
            plt.show()
            
        break

In [ ]:
with torch.no_grad():
    for data in val_dataloader:
        pose_img = data[0]
        pose_img = pose_img.permute(0, 3, 1, 2)
        pose_img = pose_img.to("cuda").to(torch.float32)
        input_emb = data[1]
        input_emb = input_emb.to("cuda").to(torch.float32)
        output_emb = data[2]
        output_emb = output_emb.to("cuda").to(torch.float32)
        
        features, _ = net.encoder(pose_img)
        
        reconstructed_data = net.decoder_pose_reconstructor(features)
        
        reconstructed_data = reconstructed_data.permute(0, 2, 3, 1).cpu().numpy()
        orig_data = pose_img.permute(0, 2, 3, 1).cpu().numpy()
        
        for i in range(reconstructed_data.shape[0]):
            plt.imshow(orig_data[i, :, :, 0])
            plt.show()
            plt.imshow(reconstructed_data[i, :, :, 0])
            plt.show()
            
        break